# [VGG]

*KU LeeDongGyu*

### Contents

1. Almost Original VGG19 & Data Import
```
1) Data Import
2) Data Augmentation
3) Almost Original VGG19
```
2. For Size = 64,
```
1) My VGG11 (Pretraining)
2) My VGG16 (Pretrained)
3) My VGG19 (Pretrained)
4) My VGG11 (Convergence Speed Comparison)
```
3. For Size = 48,
```
1) My VGG11 (Pretraining)
2) My VGG16 (Pretrained)
3) My VGG19 (Pretrained)
4) My VGG11 (Convergence Speed Comparison)
```
4. For Size = 48, No Early Stopping
```
1) Epoch = 50
      (1) My VGG11
      (2) My VGG16
      (3) My VGG19
2) Epoch = 100
      (1) My VGG11
      (2) My VGG16
      (3) My VGG19
```
5. Model Save & Import

### Install Packages

In [1]:
pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 36kB/s 
     |████████████████████████████████| 3.9MB 41.9MB/s 
     |████████████████████████████████| 450kB 48.4MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


### Module

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [5]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [0]:
from lrn import LRN
from f1score import macro_f1score,weighted_f1score

In [0]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_file, to_categorical

In [9]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/project'

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
tf.__version__

'2.1.0'

In [84]:
tf.test.gpu_device_name()

'/device:GPU:0'

## 1. Almost Original VGG19 & Data Import
---

### 1) Data Import

In [0]:
# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
# data handling
x_train = np.array(x_train).reshape([-1,48,48,3]) 
x_valid = np.array(x_valid).reshape([-1,48,48,3]) 
x_test = np.array(x_test).reshape([-1,48,48,3]) 

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
# uint는 부호없는 정수로, 타입을 바꿔줘야함!
size = 64

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,3],dtype="float32")
for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
x_train = x_train / 255

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,3])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,3],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
 x_valid = x_valid / 255

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,3])
x_test_zoom = np.zeros([x_test.shape[0],size,size,3],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) / 255

In [0]:
x_test = x_test / 255

### 2) Data Augmentation

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=[0.9,1.0])

In [0]:
datagen_val = ImageDataGenerator()

In [0]:
xy_valid_zoom_gen = datagen_val.flow(x_valid_zoom,y_valid,batch_size=128)
xy_valid_gen = datagen_val.flow(x_valid,y_valid,batch_size=128)

### 3) Almost Original VGG19

In [0]:
# VGG19 모형과 관련된 초기값이 저장되어 있는 링크. 접속시 바로 다운됨. FC층을 어떻게 처리할 건지에 따라 두가지 초기값들이 있음.
# 이 값은 ImageNet 데이터를 가지고 학습한 초기값임.
# 내가 쓸 모형은 이 초기값이 필요 없기 때문에, 내가 초기값을 뽑아내는 작업을 해야한다.

# 다시한번 얘기하지만, 아래의 두 변수는 내가 아래의 코드에서 쓸 변수는 아님. 나중의 혹시모를 상황을 대비해 남겨놓음.
WEIGHTS_PATH = ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5')
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

# -*- coding: utf-8 -*-
'''VGG19 model for Keras.
# Reference:
- [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)
'''
('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5')
"""Instantiates the VGG19 architecture.
 Optionally loads weights pre-trained on ImageNet. Note that when using TensorFlow,
 for best performance you should set
 `image_data_format="channels_last"` in your Keras config
 at ~/.keras/keras.json.
 The model and the weights are compatible with both
 TensorFlow and Theano. The data format
 convention used by the model is the one
 specified in your Keras config file.

 # Arguments
     include_top: whether to include the 3 fully-connected
         layers at the top of the network.
     weights: one of `None` (random initialization)
         or "imagenet" (pre-training on ImageNet).
     input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
         to use as image input for the model.
     input_shape: optional shape tuple, only to be specified
         if `include_top` is False (otherwise the input shape
         has to be `(224, 224, 3)` (with `channels_last` data format)
         or `(3, 224, 244)` (with `channels_first` data format).
         It should have exactly 3 inputs channels,
         and width and height should be no smaller than 48.
         E.g. `(200, 200, 3)` would be one valid value.
     pooling: Optional pooling mode for feature extraction
         when `include_top` is `False`.
         - `None` means that the output of the model will be
             the 4D tensor output of the
             last convolutional layer.
         - `avg` means that global average pooling
             will be applied to the output of the
             last convolutional layer, and thus
             the output of the model will be a 2D tensor.
         - `max` means that global max pooling will
             be applied.
     classes: optional number of classes to classify images
         into, only to be specified if `include_top` is True, and
         if no `weights` argument is specified.
 # Returns
     A Keras model instance.
 """


'Instantiates the VGG19 architecture.\n Optionally loads weights pre-trained on ImageNet. Note that when using TensorFlow,\n for best performance you should set\n `image_data_format="channels_last"` in your Keras config\n at ~/.keras/keras.json.\n The model and the weights are compatible with both\n TensorFlow and Theano. The data format\n convention used by the model is the one\n specified in your Keras config file.\n\n # Arguments\n     include_top: whether to include the 3 fully-connected\n         layers at the top of the network.\n     weights: one of `None` (random initialization)\n         or "imagenet" (pre-training on ImageNet).\n     input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)\n         to use as image input for the model.\n     input_shape: optional shape tuple, only to be specified\n         if `include_top` is False (otherwise the input shape\n         has to be `(224, 224, 3)` (with `channels_last` data format)\n         or `(3, 224, 244)` (with 

In [0]:
# VGG19를 최대한 논문에 가깝게 맞춰 모형작성.
# 또한, Data Augmentation은 컴퓨터 성능의 한계로 하지 않음.

def VGG19(input_shape=(224, 224, 3), classes=7, include_top=True, pooling=None, weights=None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    if weights == 'imagenet': # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH, cache_subdir='models', file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP, cache_subdir='models',
                                    file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path)  # 경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model

In [0]:
model = VGG19(input_shape=(224, 224, 3), classes=7)

In [0]:
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## 2. For Size = 64,
---

### 1) My VGG11 (Pretraining)

In [0]:
# 논문에서 VGG19 의 FC층은 VGG11로 pretraining하여 얻은 초기치로 설정하였음.
# 기존의 VGG11을 개조
# Data Augmentation은 컴퓨터 성능의 한계로 못하기 때문에 변형함.

# 주의 !!!!기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!!!

# 다음의 절차로 모형을 개조한다.

# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.

def VGG11(input_shape=(64,64,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu',name='fc1')(x)
        x = Dense(256, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11_Pretraining')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
#내 데이터 맞춤형 모형
model = VGG11(input_shape=(64, 64, 3), classes=7, include_top=True,pooling=None, weights = None)

In [31]:
model.summary()

Model: "Vgg11_Pretraining"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         224       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 16, 16, 32)  

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [33]:
model.fit(datagen.flow(x_train_zoom,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_zoom_gen, validation_steps=len(x_valid_zoom)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 41s 184ms/step - loss: 1.8169 - accuracy: 0.2504 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8546 - val_accuracy: 0.2536 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 36s 158ms/step - loss: 1.7427 - accuracy: 0.2892 - macro_f1score: 0.0240 - weighted_f1score: 0.0053 - val_loss: 1.6695 - val_accuracy: 0.3692 - val_macro_f1score: 0.0783 - val_weighted_f1score: 0.0176
Epoch 3/100
225/224 [==============================] - 36s 159ms/step - loss: 1.5996 - accuracy: 0.3667 - macro_f1score: 0.1162 - weighted_f1score: 0.0231 - val_loss: 1.5611 - val_accuracy: 0.4043 - val_macro_f1score: 0.1568 - val_weighted_f1score: 0.0311
Epoch 4/100
225/224 [==============================] - 35s 157ms/step - loss: 1.5066 - accuracy: 0.4094 - ma

In [34]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 1s 160us/sample - loss: 1.0969 - accuracy: 0.5766 - macro_f1score: 0.4014 - weighted_f1score: 0.0682

Accuracy: 0.5766, Macro F1 Score: 0.4014, Weighted F1 Score: 0.0682


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

In [36]:
W

[array([[[[ 0.20407596, -0.0076126 , -0.31642285, -0.14033513,
            0.2834546 ,  0.17670147,  0.18333197, -0.00525202],
          [-0.1222116 , -0.00253352, -0.1462644 , -0.12128124,
            0.26775965,  0.1779506 ,  0.20969276,  0.0413659 ],
          [-0.07989866,  0.27928892, -0.27633652, -0.0575515 ,
            0.09372558,  0.2992469 , -0.07647823,  0.14722785]],
 
         [[ 0.02342825, -0.07014901,  0.00285076, -0.23947963,
           -0.15778463, -0.07801206, -0.15598541, -0.08965971],
          [ 0.13409467,  0.20458865, -0.09732905, -0.14803115,
           -0.16993253, -0.14117503, -0.15697259,  0.06676271],
          [ 0.34458554,  0.09123912,  0.06526193,  0.17850512,
           -0.00704311, -0.0696204 , -0.02253206, -0.24609652]],
 
         [[ 0.07324352, -0.13435145, -0.01040823,  0.06470463,
            0.17856698,  0.0033257 , -0.06190493, -0.23788494],
          [ 0.22493629, -0.11870138, -0.18789716, -0.18417086,
           -0.22343037, -0.04021792, -0.15

In [37]:
model.weights # 참고. get_weights()와 weights는 다른 매서드임을 알자.

[<tf.Variable 'block1_conv1_1/kernel:0' shape=(3, 3, 3, 8) dtype=float32, numpy=
 array([[[[ 0.20407596, -0.0076126 , -0.31642285, -0.14033513,
            0.2834546 ,  0.17670147,  0.18333197, -0.00525202],
          [-0.1222116 , -0.00253352, -0.1462644 , -0.12128124,
            0.26775965,  0.1779506 ,  0.20969276,  0.0413659 ],
          [-0.07989866,  0.27928892, -0.27633652, -0.0575515 ,
            0.09372558,  0.2992469 , -0.07647823,  0.14722785]],
 
         [[ 0.02342825, -0.07014901,  0.00285076, -0.23947963,
           -0.15778463, -0.07801206, -0.15598541, -0.08965971],
          [ 0.13409467,  0.20458865, -0.09732905, -0.14803115,
           -0.16993253, -0.14117503, -0.15697259,  0.06676271],
          [ 0.34458554,  0.09123912,  0.06526193,  0.17850512,
           -0.00704311, -0.0696204 , -0.02253206, -0.24609652]],
 
         [[ 0.07324352, -0.13435145, -0.01040823,  0.06470463,
            0.17856698,  0.0033257 , -0.06190493, -0.23788494],
          [ 0.22493629, 

In [38]:
# 형태는 이렇게 생김
np.array(W).shape

(22,)

### 2) My VGG16 (Pretrained)

In [0]:
# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.


def VGG16(input_shape=(64,64,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
model = VGG16(input_shape=(64, 64, 3), classes=7, include_top=True,pooling=None, weights = None)

In [41]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [43]:
model.fit(datagen.flow(x_train_zoom,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_zoom_gen, validation_steps=len(x_valid_zoom)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 36s 160ms/step - loss: 1.8086 - accuracy: 0.2496 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8744 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 35s 154ms/step - loss: 1.7427 - accuracy: 0.2839 - macro_f1score: 0.0215 - weighted_f1score: 0.0026 - val_loss: 1.7383 - val_accuracy: 0.3260 - val_macro_f1score: 0.0053 - val_weighted_f1score: 7.8795e-04
Epoch 3/100
225/224 [==============================] - 35s 154ms/step - loss: 1.6157 - accuracy: 0.3614 - macro_f1score: 0.1100 - weighted_f1score: 0.0207 - val_loss: 1.5534 - val_accuracy: 0.4199 - val_macro_f1score: 0.1524 - val_weighted_f1score: 0.0289
Epoch 4/100
225/224 [==============================] - 35s 154ms/step - loss: 1.5183 - accuracy: 0.4059 

In [44]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 142us/sample - loss: 1.1246 - accuracy: 0.5889 - macro_f1score: 0.4036 - weighted_f1score: 0.0678

Accuracy: 0.5889, Macro F1 Score: 0.4036, Weighted F1 Score: 0.0678


### 3) My VGG19 (Pretrained)

In [0]:
# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.


def VGG19(input_shape=(64,64,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
#내 데이터 맞춤형 모형
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(64, 64, 3), classes=7, include_top=True,pooling=None, weights = None)


In [47]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [49]:
model.fit(datagen.flow(x_train_zoom,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_zoom_gen, validation_steps=len(x_valid_zoom)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 36s 162ms/step - loss: 1.7821 - accuracy: 0.2614 - macro_f1score: 0.0061 - weighted_f1score: 6.9450e-04 - val_loss: 1.7594 - val_accuracy: 0.3015 - val_macro_f1score: 0.0300 - val_weighted_f1score: 0.0035
Epoch 2/100
225/224 [==============================] - 35s 155ms/step - loss: 1.6484 - accuracy: 0.3422 - macro_f1score: 0.0870 - weighted_f1score: 0.0157 - val_loss: 1.5930 - val_accuracy: 0.3906 - val_macro_f1score: 0.1479 - val_weighted_f1score: 0.0292
Epoch 3/100
225/224 [==============================] - 35s 153ms/step - loss: 1.5287 - accuracy: 0.3982 - macro_f1score: 0.1461 - weighted_f1score: 0.0279 - val_loss: 1.4913 - val_accuracy: 0.4372 - val_macro_f1score: 0.1676 - val_weighted_f1score: 0.0326
Epoch 4/100
225/224 [==============================] - 35s 155ms/step - loss: 1.4516 - accuracy: 0.4300 - macro_f1score:

In [50]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 1s 147us/sample - loss: 1.1117 - accuracy: 0.5828 - macro_f1score: 0.4091 - weighted_f1score: 0.0695

Accuracy: 0.5828, Macro F1 Score: 0.4091, Weighted F1 Score: 0.0695


### 4) My VGG11 (Convergence Speed Comparison)

In [0]:
# Vgg11에서, pretraining 된 fc층을 가지고 VGG 11을 재학습 시킬때, 얼마나 적은 epoch만에 수렴하는지를 보기 위해 만들었다.


# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.

def VGG11_init(input_shape=(64,64,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11_Pretrained')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
model = VGG11_init(input_shape=(64, 64, 3), classes=7, include_top=True,pooling=None, weights = None)

In [53]:
model.summary()

Model: "Vgg11_Pretrained"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         224       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 16, 16, 32)   

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [55]:
model.fit(datagen.flow(x_train_zoom,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_zoom_gen, validation_steps=len(x_valid_zoom)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 35s 158ms/step - loss: 1.7632 - accuracy: 0.2705 - macro_f1score: 0.0097 - weighted_f1score: 0.0011 - val_loss: 1.6794 - val_accuracy: 0.3697 - val_macro_f1score: 0.0633 - val_weighted_f1score: 0.0100
Epoch 2/100
225/224 [==============================] - 34s 152ms/step - loss: 1.5749 - accuracy: 0.3792 - macro_f1score: 0.1248 - weighted_f1score: 0.0237 - val_loss: 1.5179 - val_accuracy: 0.4177 - val_macro_f1score: 0.1683 - val_weighted_f1score: 0.0327
Epoch 3/100
225/224 [==============================] - 34s 153ms/step - loss: 1.4681 - accuracy: 0.4253 - macro_f1score: 0.1660 - weighted_f1score: 0.0313 - val_loss: 1.4500 - val_accuracy: 0.4539 - val_macro_f1score: 0.1841 - val_weighted_f1score: 0.0365
Epoch 4/100
225/224 [==============================] - 34s 153ms/step - loss: 1.3793 - accuracy: 0.4630 - macro_f1score: 0.2

In [56]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 120us/sample - loss: 1.0887 - accuracy: 0.5878 - macro_f1score: 0.4361 - weighted_f1score: 0.0717

Accuracy: 0.5878, Macro F1 Score: 0.4361, Weighted F1 Score: 0.0717


## 3. For Size =48,
---

### 1) My VGG11 (Pretraining)

In [0]:
# 오리지널 데이터에 대해 다루어본다.

def VGG11(input_shape=(48,48,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((3, 3), strides=(3, 3), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu',name='fc1')(x)
        x = Dense(256, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11_Pretraining')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
#내 데이터 맞춤형 모형
model = VGG11(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [59]:
model.summary()

Model: "Vgg11_Pretraining"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         224       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 12, 12, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 12, 12, 32)  

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [61]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 28s 126ms/step - loss: 1.8133 - accuracy: 0.2516 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8679 - val_accuracy: 0.2519 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 27s 122ms/step - loss: 1.7794 - accuracy: 0.2607 - macro_f1score: 5.5400e-04 - weighted_f1score: 5.8114e-05 - val_loss: 1.8071 - val_accuracy: 0.2845 - val_macro_f1score: 0.0018 - val_weighted_f1score: 2.1687e-04
Epoch 3/100
225/224 [==============================] - 27s 122ms/step - loss: 1.6926 - accuracy: 0.3138 - macro_f1score: 0.0499 - weighted_f1score: 0.0087 - val_loss: 1.6125 - val_accuracy: 0.3842 - val_macro_f1score: 0.1314 - val_weighted_f1score: 0.0246
Epoch 4/100
225/224 [==============================] - 27s 122ms/step - loss: 1.5602 - accuracy:

In [62]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 0s 130us/sample - loss: 1.1217 - accuracy: 0.5730 - macro_f1score: 0.3803 - weighted_f1score: 0.0648

Accuracy: 0.5730, Macro F1 Score: 0.3803, Weighted F1 Score: 0.0648


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

In [64]:
W

[array([[[[ 0.00603382,  0.02733675, -0.0172347 ,  0.09976361,
            0.03591269,  0.06154356, -0.27886167,  0.1747594 ],
          [ 0.00335163, -0.21710117,  0.22996181,  0.1251715 ,
           -0.10697839,  0.04452933,  0.0711987 ,  0.24787195],
          [-0.07506111, -0.25114205,  0.15309526, -0.13634756,
           -0.23240252, -0.12852968, -0.19172572,  0.19764704]],
 
         [[-0.13800357, -0.18231395,  0.20344728, -0.17242506,
            0.11104856,  0.23513095,  0.2379421 ,  0.11059583],
          [-0.01358065, -0.20922793, -0.12049043, -0.09114791,
            0.10302974,  0.17773746,  0.13409042, -0.02881867],
          [ 0.09041117,  0.12680635, -0.14835255,  0.17584477,
           -0.04743635, -0.10408151,  0.18776798, -0.00561239]],
 
         [[ 0.3224108 ,  0.17165905,  0.19499454,  0.1848479 ,
           -0.1850659 ,  0.09877757,  0.12642926,  0.09629154],
          [-0.08099049, -0.19010457, -0.24314374,  0.13739224,
           -0.23073164, -0.15595315, -0.10

In [65]:
model.weights # 참고. get_weights()와 weights는 다른 매서드임을 알자.

[<tf.Variable 'block1_conv1_5/kernel:0' shape=(3, 3, 3, 8) dtype=float32, numpy=
 array([[[[ 0.00603382,  0.02733675, -0.0172347 ,  0.09976361,
            0.03591269,  0.06154356, -0.27886167,  0.1747594 ],
          [ 0.00335163, -0.21710117,  0.22996181,  0.1251715 ,
           -0.10697839,  0.04452933,  0.0711987 ,  0.24787195],
          [-0.07506111, -0.25114205,  0.15309526, -0.13634756,
           -0.23240252, -0.12852968, -0.19172572,  0.19764704]],
 
         [[-0.13800357, -0.18231395,  0.20344728, -0.17242506,
            0.11104856,  0.23513095,  0.2379421 ,  0.11059583],
          [-0.01358065, -0.20922793, -0.12049043, -0.09114791,
            0.10302974,  0.17773746,  0.13409042, -0.02881867],
          [ 0.09041117,  0.12680635, -0.14835255,  0.17584477,
           -0.04743635, -0.10408151,  0.18776798, -0.00561239]],
 
         [[ 0.3224108 ,  0.17165905,  0.19499454,  0.1848479 ,
           -0.1850659 ,  0.09877757,  0.12642926,  0.09629154],
          [-0.08099049, 

In [66]:
# 형태는 이렇게 생김
np.array(W).shape

(22,)

### 2) My VGG16 (Pretrained)

In [0]:
def VGG16(input_shape=(48,48,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
model = VGG16(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [87]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [89]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 30s 132ms/step - loss: 1.8073 - accuracy: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8556 - val_accuracy: 0.2602 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 28s 126ms/step - loss: 1.7501 - accuracy: 0.2808 - macro_f1score: 0.0255 - weighted_f1score: 0.0035 - val_loss: 1.7189 - val_accuracy: 0.3433 - val_macro_f1score: 0.0719 - val_weighted_f1score: 0.0123
Epoch 3/100
225/224 [==============================] - 29s 127ms/step - loss: 1.6152 - accuracy: 0.3518 - macro_f1score: 0.1143 - weighted_f1score: 0.0219 - val_loss: 1.5940 - val_accuracy: 0.3764 - val_macro_f1score: 0.1508 - val_weighted_f1score: 0.0284
Epoch 4/100
225/224 [==============================] - 29s 127ms/step - loss: 1.5425 - accuracy: 0.3840 - ma

In [90]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 130us/sample - loss: 1.1879 - accuracy: 0.5365 - macro_f1score: 0.3312 - weighted_f1score: 0.0585

Accuracy: 0.5365, Macro F1 Score: 0.3312, Weighted F1 Score: 0.0585


### 3) My VGG19 (Pretrained)

In [0]:
def VGG19(input_shape=(48,48,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((3, 3), strides=(3, 3), name='block5_pool')(x) 
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.   

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
#내 데이터 맞춤형 모형
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [93]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         224       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [100]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 30s 131ms/step - loss: 1.6886 - accuracy: 0.3150 - macro_f1score: 0.0560 - weighted_f1score: 0.0086 - val_loss: 1.7725 - val_accuracy: 0.3112 - val_macro_f1score: 0.0905 - val_weighted_f1score: 0.0132
Epoch 2/100
225/224 [==============================] - 29s 131ms/step - loss: 1.6353 - accuracy: 0.3437 - macro_f1score: 0.1049 - weighted_f1score: 0.0195 - val_loss: 1.6559 - val_accuracy: 0.3642 - val_macro_f1score: 0.1391 - val_weighted_f1score: 0.0258
Epoch 3/100
225/224 [==============================] - 29s 130ms/step - loss: 1.5815 - accuracy: 0.3705 - macro_f1score: 0.1291 - weighted_f1score: 0.0245 - val_loss: 1.6197 - val_accuracy: 0.3823 - val_macro_f1score: 0.1340 - val_weighted_f1score: 0.0235
Epoch 4/100
225/224 [==============================] - 29s 130ms/step - loss: 1.5352 - accuracy: 0.3977 - macro_f1score: 0.1

In [101]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 1s 141us/sample - loss: 1.3594 - accuracy: 0.4746 - macro_f1score: 0.2274 - weighted_f1score: 0.0419

Accuracy: 0.4746, Macro F1 Score: 0.2274, Weighted F1 Score: 0.0419


### 4) My VGG11 (Convergence Speed Comparison)

In [0]:
# Vgg11에서, pretraining 된 fc층을 가지고 VGG 11을 재학습 시킬때, 얼마나 적은 epoch만에 수렴하는지를 보기 위해 만들었다.

def VGG11_init(input_shape=(48,48,3), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11_Pretrained')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
model = VGG11_init(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [104]:
model.summary()

Model: "Vgg11_Pretrained"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         224       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 12, 12, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 12, 12, 32)   

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [106]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=100, callbacks=[early_stopping]) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 29s 129ms/step - loss: 1.7723 - accuracy: 0.2641 - macro_f1score: 0.0057 - weighted_f1score: 6.8730e-04 - val_loss: 1.7416 - val_accuracy: 0.3263 - val_macro_f1score: 0.0240 - val_weighted_f1score: 0.0030
Epoch 2/100
225/224 [==============================] - 28s 127ms/step - loss: 1.6160 - accuracy: 0.3587 - macro_f1score: 0.1034 - weighted_f1score: 0.0198 - val_loss: 1.6015 - val_accuracy: 0.3948 - val_macro_f1score: 0.1127 - val_weighted_f1score: 0.0215
Epoch 3/100
225/224 [==============================] - 29s 128ms/step - loss: 1.5221 - accuracy: 0.4020 - macro_f1score: 0.1432 - weighted_f1score: 0.0279 - val_loss: 1.4894 - val_accuracy: 0.4352 - val_macro_f1score: 0.1604 - val_weighted_f1score: 0.0310
Epoch 4/100
225/224 [==============================] - 29s 127ms/step - loss: 1.4647 - accuracy: 0.4312 - macro_f1score:

In [107]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 124us/sample - loss: 1.1556 - accuracy: 0.5641 - macro_f1score: 0.3822 - weighted_f1score: 0.0652

Accuracy: 0.5641, Macro F1 Score: 0.3822, Weighted F1 Score: 0.0652


## 4. For Size = 48, No Early Stopping
---

### 1) Epoch = 50

#### (1) My VGG11

In [0]:
############# 이 아래는 early stopping 없이 진행 #######################

In [0]:
# VGG 11 - pretraining
model = VGG11(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [111]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=50) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/50
225/224 [==============================] - 30s 132ms/step - loss: 1.8179 - accuracy: 0.2487 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8694 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
225/224 [==============================] - 28s 126ms/step - loss: 1.7497 - accuracy: 0.2820 - macro_f1score: 0.0292 - weighted_f1score: 0.0058 - val_loss: 1.6380 - val_accuracy: 0.3831 - val_macro_f1score: 0.1331 - val_weighted_f1score: 0.0248
Epoch 3/50
225/224 [==============================] - 28s 126ms/step - loss: 1.5519 - accuracy: 0.3819 - macro_f1score: 0.1420 - weighted_f1score: 0.0275 - val_loss: 1.5473 - val_accuracy: 0.4107 - val_macro_f1score: 0.1544 - val_weighted_f1score: 0.0303
Epoch 4/50
225/224 [==============================] - 29s 127ms/step - loss: 1.4762 - accuracy: 0.4204 - macro_

In [112]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 117us/sample - loss: 1.1078 - accuracy: 0.5856 - macro_f1score: 0.4504 - weighted_f1score: 0.0723

Accuracy: 0.5856, Macro F1 Score: 0.4504, Weighted F1 Score: 0.0723


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

#### (2) My VGG16

In [0]:
# VGG 16 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG16(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [116]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=50) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/50
225/224 [==============================] - 31s 136ms/step - loss: 1.8002 - accuracy: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8273 - val_accuracy: 0.2666 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
225/224 [==============================] - 29s 130ms/step - loss: 1.7181 - accuracy: 0.2959 - macro_f1score: 0.0302 - weighted_f1score: 0.0040 - val_loss: 1.6887 - val_accuracy: 0.3566 - val_macro_f1score: 0.0798 - val_weighted_f1score: 0.0144
Epoch 3/50
225/224 [==============================] - 29s 129ms/step - loss: 1.5853 - accuracy: 0.3725 - macro_f1score: 0.1236 - weighted_f1score: 0.0235 - val_loss: 1.5754 - val_accuracy: 0.4107 - val_macro_f1score: 0.1649 - val_weighted_f1score: 0.0307
Epoch 4/50
225/224 [==============================] - 29s 130ms/step - loss: 1.4993 - accuracy: 0.4146 - macro_

In [117]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 138us/sample - loss: 1.0883 - accuracy: 0.6042 - macro_f1score: 0.4372 - weighted_f1score: 0.0734

Accuracy: 0.6042, Macro F1 Score: 0.4372, Weighted F1 Score: 0.0734


#### (3) My VGG19

In [0]:
# VGG 19 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [133]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=50) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/50
225/224 [==============================] - 31s 136ms/step - loss: 1.8134 - accuracy: 0.2499 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8832 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
225/224 [==============================] - 30s 134ms/step - loss: 1.8125 - accuracy: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8817 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/50
225/224 [==============================] - 30s 133ms/step - loss: 1.8110 - accuracy: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8795 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/50
225/224 [==============================] - 30s 133ms/step - loss: 1.

In [134]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 145us/sample - loss: 1.8111 - accuracy: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


### 2) Epoch = 100

#### (1) My VGG11

In [0]:
# VGG 11 - pretraining
model = VGG11(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [137]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=100) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 30s 134ms/step - loss: 1.8193 - accuracy: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8814 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 30s 131ms/step - loss: 1.7859 - accuracy: 0.2629 - macro_f1score: 0.0043 - weighted_f1score: 4.8646e-04 - val_loss: 1.8012 - val_accuracy: 0.2781 - val_macro_f1score: 0.0401 - val_weighted_f1score: 0.0047
Epoch 3/100
225/224 [==============================] - 29s 130ms/step - loss: 1.6905 - accuracy: 0.3067 - macro_f1score: 0.0617 - weighted_f1score: 0.0086 - val_loss: 1.6502 - val_accuracy: 0.3692 - val_macro_f1score: 0.1262 - val_weighted_f1score: 0.0227
Epoch 4/100
225/224 [==============================] - 30s 131ms/step - loss: 1.5707 - accuracy: 0.3756 

In [138]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 115us/sample - loss: 1.0850 - accuracy: 0.6056 - macro_f1score: 0.4916 - weighted_f1score: 0.0793

Accuracy: 0.6056, Macro F1 Score: 0.4916, Weighted F1 Score: 0.0793


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

#### (2) My VGG16

In [0]:
# VGG 16 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG16(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [142]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=100) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 31s 139ms/step - loss: 1.8034 - accuracy: 0.2496 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8576 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 30s 133ms/step - loss: 1.7867 - accuracy: 0.2525 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8414 - val_accuracy: 0.2625 - val_macro_f1score: 0.0249 - val_weighted_f1score: 0.0029
Epoch 3/100
225/224 [==============================] - 30s 132ms/step - loss: 1.7312 - accuracy: 0.2858 - macro_f1score: 0.0210 - weighted_f1score: 0.0023 - val_loss: 1.7283 - val_accuracy: 0.3232 - val_macro_f1score: 0.0884 - val_weighted_f1score: 0.0132
Epoch 4/100
225/224 [==============================] - 30s 132ms/step - loss: 1.6153 - accuracy: 0.3

In [143]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 141us/sample - loss: 1.0926 - accuracy: 0.6081 - macro_f1score: 0.4688 - weighted_f1score: 0.0759

Accuracy: 0.6081, Macro F1 Score: 0.4688, Weighted F1 Score: 0.0759


#### (3) My VGG19

In [0]:
# VGG 19 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 3), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [146]:
model.fit(datagen.flow(x_train,y_train,batch_size=128), steps_per_epoch=len(x_train)/128, validation_data= xy_valid_gen, validation_steps=len(x_valid)/128, epochs=100) 

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 224.203125 steps, validate for 28.0390625 steps
Epoch 1/100
225/224 [==============================] - 32s 144ms/step - loss: 1.8127 - accuracy: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8784 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
225/224 [==============================] - 31s 136ms/step - loss: 1.8111 - accuracy: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8788 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
225/224 [==============================] - 31s 139ms/step - loss: 1.8112 - accuracy: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8808 - val_accuracy: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
225/224 [==============================] - 31s 137ms/step - loss

In [147]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 152us/sample - loss: 1.8110 - accuracy: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


## 5. Model Save & Import
---

In [0]:
#참고.
model.save('CNN model/VGG19.h5') # 모델 저장

In [0]:
model.save_weights('CNN model/VGG19_weights.h5') # 가중치 저장

In [0]:
model2 = load_model('CNN model/VGG19.h5',custom_objects={"macro_f1score": macro_f1score,"weighted_f1score":weighted_f1score}) # 모델 불러오기  custom_objects={"":} 로 정의했던 측도들 불러온다.